In [ ]:
!pip install wordcloud

In [ ]:
!pip install -U spacy

     |████████████████████████████████| 6.2 MB 4.2 MB/s 
     |████████████████████████████████| 42 kB 1.1 MB/s 
     |████████████████████████████████| 181 kB 46.0 MB/s 
     |████████████████████████████████| 660 kB 48.7 MB/s 
     |████████████████████████████████| 10.1 MB 42.4 MB/s 
     |████████████████████████████████| 457 kB 49.6 MB/s 
     |████████████████████████████████| 58 kB 4.8 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.2.0
    Uninstalling typing-extensions-4.2.0:
      Successfully uninstalled typing-extensions-4.2.0
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: smart-open
    Found existing installation: smart-open 6.0.0
    U

In [ ]:
import sklearn
print(sklearn.__version__)

1.0.2


In [ ]:
import re 
import string
import numpy as np 
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin
from sklearn.metrics import accuracy_score,  plot_confusion_matrix,classification_report, confusion_matrix
from wordcloud import WordCloud

import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

In [ ]:
import numpy as np 
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt
%matplotlib inline
pd.set_option('display.max_columns', None)

import re 
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

import joblib
from nltk.tokenize import RegexpTokenizer
import nltk
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, recall_score, plot_confusion_matrix 
from sklearn.metrics import roc_curve, precision_recall_curve, auc, make_scorer, recall_score, accuracy_score, precision_score

from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin
from imblearn.combine import SMOTETomek
import string
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from wordcloud import WordCloud

import random
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/fake_job_postings.csv') 

In [ ]:
df.isnull().sum()

job_id                     0
title                      0
location                 346
department             11547
salary_range           15012
company_profile         3308
description                1
requirements            2695
benefits                7210
telecommuting              0
has_company_logo           0
has_questions              0
employment_type         3471
required_experience     7050
required_education      8105
industry                4903
function                6455
fraudulent                 0
dtype: int64

In [ ]:
columns=['job_id','telecommuting','has_company_logo','has_questions','salary_range','employment_type']
for colu in columns:
    del df[colu]

In [ ]:
df.fillna('', inplace=True)

In [ ]:
exp= dict (df.required_experience.value_counts())
del exp['']

In [ ]:
def split(location):
    l= location.split(',')
    return l[0]
df['country']= df.location.apply(split)

In [ ]:
countr=dict(df.country.value_counts()[:14])
del countr['']
countr

{'AU': 214,
 'BE': 117,
 'CA': 457,
 'DE': 383,
 'GB': 2384,
 'GR': 940,
 'IE': 114,
 'IN': 276,
 'NL': 127,
 'NZ': 333,
 'PH': 132,
 'SG': 80,
 'US': 10656}

In [ ]:
edu= dict(df.required_education.value_counts()[:7])
del edu['']
edu

{'Associate Degree': 274,
 "Bachelor's Degree": 5145,
 'Certification': 170,
 'High School or equivalent': 2080,
 "Master's Degree": 416,
 'Unspecified': 1397}

In [ ]:
df['text']=df['title']+' '+df['location']+' '+df['company_profile']+' '+df['description']+' '+df['requirements']+' '+df['benefits']

df.text.fillna(' ', inplace=True)

fraudjobs_text = df[df.fraudulent==0].text

In [ ]:
import nltk
nltk.download('stopwords')

stop_words = set(stopwords.words("english"))
default_stemmer = PorterStemmer()
default_stopwords = stopwords.words('english')
default_tokenizer=RegexpTokenizer(r"\w+")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
df['text']=df['title']+' '+df['location']+' '+df['company_profile']+' '+df['description']+' '+df['requirements']+' '+df['benefits']

df.text.fillna(' ', inplace=True)

fraudjobs_text = df[df.fraudulent==1].text

In [ ]:
#merge the title, location, profile, description, requirements into a mega text column

df['text']=df['title']+' '+df['location']+' '+df['company_profile']+' '+df['description']+' '+df['requirements']+' '+df['benefits']

df = df[['text','fraudulent']]

In [ ]:
!pip install spacy && python -m spacy download en

⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     |████████████████████████████████| 12.8 MB 4.3 MB/s 
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uninstalled en-core-web-sm-2.2.5
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
# spacy tokenizer

# Create our list of punctuation marks
punctuations = string.punctuation

# Create our list of stopwords
nlp = spacy.load("en_core_web_sm")
stop_words = spacy.lang.en.stop_words.STOP_WORDS

# Load English tokenizer, tagger, parser, NER and word vectors
parser = English()

def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = parser(sentence)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

# Custom cleanning class
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        # Cleaning Text
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}

def clean_text(text):
    # Removing spaces and converting text into lowercase
    # TBD

    return text.strip().lower()

In [ ]:
df['text'] =df['text'].apply(clean_text)

In [ ]:
cv =TfidfVectorizer(max_features =100)
x=cv.fit_transform(df['text'])
df1=pd.DataFrame(x.toarray(), columns=cv.get_feature_names())
df.drop(["text"], axis=1,inplace=True)
main_df= pd.concat([df1,df], axis =1)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
Y=main_df.iloc[:,-1]
X=main_df.iloc[:,:-1]

X_train,X_test, y_train, y_test =train_test_split(X,Y, test_size=0.3)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(12516, 100)
(12516,)
(5364, 100)
(5364,)


In [ ]:
from sklearn.preprocessing import StandardScaler
Scaler_X = StandardScaler()
X_train = Scaler_X.fit_transform(X_train)
X_test = Scaler_X.transform(X_test)

**Smote**

In [ ]:
 from collections import Counter
from imblearn.over_sampling import SMOTE

counter = Counter(y_train)
print('Before',counter)
# oversampling the train dataset using SMOTE
smt = SMOTE()
#X_train, y_train = smt.fit_resample(X_train, y_train)
X_train_sm, y_train_sm = smt.fit_resample(X_train, y_train)

counter = Counter(y_train_sm)
print('After',counter)

Before Counter({0: 11889, 1: 627})
After Counter({0: 11889, 1: 11889})


**KNN**

In [ ]:
import time
import warnings
import pandas as pd
from sklearn.metrics import auc
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve
from sklearn.dummy import DummyClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
warnings.filterwarnings("ignore")

In [ ]:
start_time = time.time()
knn = KNeighborsClassifier()
n_neighbors = [i for i in range(2, 6)]
param_dist = {"n_neighbors": n_neighbors, "weights": ['distance'], "n_jobs": [-1]}

In [ ]:
grid_search = GridSearchCV(knn, param_grid=param_dist, scoring='f1', cv=5)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_jobs': [-1], 'n_neighbors': [2, 3, 4, 5],
                         'weights': ['distance']},
             scoring='f1')

In [ ]:
best_knn = grid_search.best_estimator_
print(best_knn)

KNeighborsClassifier(n_jobs=-1, n_neighbors=2, weights='distance')


In [ ]:
y_predict_knn = best_knn.predict(X_test)
end_time = time.time()

In [38]:
knn_report = classification_report(y_test, y_predict_knn)
print(knn_report)
print("Execution time for KNN(SMOTE): %s min" % ((end_time - start_time)/60))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5125
           1       0.79      0.73      0.76       239

    accuracy                           0.98      5364
   macro avg       0.89      0.86      0.88      5364
weighted avg       0.98      0.98      0.98      5364

Execution time for KNN(SMOTE): 0.4750571767489115 min
